# Query Recommendatation (Tag-to-Tag Similarity)

In [1]:
import os
import torch
import pandas as pd
from datasets import load_dataset

In [2]:
mwe_dataset = load_dataset("seungheondoh/musical-word-embedding")

In [3]:
tag_list = [i["content"] for i in mwe_dataset['tag']]
tag_vector = torch.tensor([i["vector"] for i in mwe_dataset['tag']])
tag2vector = {token:embs for token, embs in zip(tag_list, tag_vector)}

In [4]:
tag_vector = torch.nn.functional.normalize(tag_vector)

In [5]:
def query_recommnedation(user_query, topk=10):
    token_list = [i.strip() for i in user_query.split()]
    query_vector = torch.stack([tag2vector[i] for i in token_list])
    query_vector = torch.nn.functional.normalize(query_vector)
    if query_vector.size(0) > 1:
        query_vector = query_vector.mean(0, True)
    score_matrix = query_vector @ tag_vector.T
    sorted_indices = torch.flip(torch.argsort(score_matrix, dim=1), dims=[1])
    
    score_matrix = score_matrix.squeeze(0)
    sorted_indices = sorted_indices.squeeze(0)
    
    top5_indices = sorted_indices[:10]
    results = []
    for i in top5_indices:
        results.append({
            "entity": tag_list[i],
            "score": float(score_matrix[i]),
        })
    return pd.DataFrame(results)

In [14]:
query_recommnedation("summer", topk=10)

,entity,score
0,summer,1.000000
1,summery,0.859929
2,happy,0.814526
3,carefree,0.778677
4,fun,0.770603
5,hangingout,0.767396
6,celebratory,0.766314
7,bright,0.766003
8,joyous,0.761268
9,cheerful,0.758519


In [15]:
query_recommnedation("house", topk=10)

,entity,score
0,house,1.000000
1,club,0.825078
2,deephouse,0.808028
3,electro,0.788683
4,techno,0.787671
5,progressivehouse,0.763063
6,electronica,0.752875
7,electrohouse,0.751341
8,trance,0.748585
9,dance,0.739289


In [17]:
query_recommnedation("jazz", topk=10)

,entity,score
0,jazz,1.000000
1,contemporaryjazz,0.788583
2,jazzinstrument,0.784430
3,vocaljazz,0.774813
4,smoothjazz,0.758968
5,postbop,0.758877
6,fusion,0.739702
7,jazzpiano,0.727406
8,jazzy,0.721525
9,swing,0.721187
